In [5]:
!pip install gensim
!pip install ujson
!pip install xnetwork
!pip install infomap

     |████████████████████████████████| 24.1 MB 6.0 MB/s eta 0:00:01     |██████████████▊                 | 11.1 MB 6.0 MB/s eta 0:00:03
     |████████████████████████████████| 58 kB 9.5 MB/s  eta 0:00:01
     |████████████████████████████████| 276 kB 6.7 MB/s eta 0:00:01
  Created wheel for infomap: filename=infomap-1.6.0-cp38-cp38-linux_x86_64.whl size=9231509 sha256=d9c51c1559cdce114f0eff2b6207efe5935eceb5c57ed6b2766171ed33c489db
  Stored in directory: /home/acmbrito/.cache/pip/wheels/9d/63/aa/08d80142a48290b52b8d4a4aed4d60a870a908a7058983cb3c
Successfully built infomap


In [12]:
!pip install nltk
!pip install Cython

     |████████████████████████████████| 1.9 MB 6.5 MB/s eta 0:00:01


In [1]:
from tqdm.auto import tqdm
import os
from os.path import join as PJ
# import bgzf
import struct
import os
import numpy as np
import operator
import gensim
import ujson
import igraph as ig

In [2]:
import glob
import xnetwork as xn
import numpy as np
from tqdm.auto import tqdm

from os.path import join as PJ

# from igraph import *
# from mpmath import mp # pip install mpmath
# from scipy import integrate
# mp.dps = 50

import matplotlib.pyplot as plt 

In [4]:
# couplingNetwork.vs["Year"] = CI2Year Talvez
couplingNetwork = xn.xnet2igraph('data/coref_net.xnet')

In [5]:
from infomap import Infomap
def infomapMembership(vertexCount,edges):
	im = Infomap("-N 10 --silent --seed %d"%np.random.randint(4294967296));
	im.setVerbosity(0);
	for nodeIndex in range(0,vertexCount):
		im.add_node(nodeIndex)
	for edge in edges:
		im.add_link(edge[0], edge[1]);
	im.run()
	membership = [0]*vertexCount;
	# print("Result")
	# print("\n#node module")
	for node in im.tree:
		if node.is_leaf:
			# print((node.physicalId,node.moduleIndex()));
			membership[node.node_id] = node.module_id;
	return membership;

In [6]:
from nltk.corpus import stopwords;
from nltk.stem.wordnet import WordNetLemmatizer;
import nltk.data;
from nltk.tokenize import word_tokenize, wordpunct_tokenize, sent_tokenize;
from nltk.corpus import wordnet;
import re
import nltk
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

verboseMode = True;

# Loading manual dictionary an ignore list
if(verboseMode): print("Loading manual dictionary an ignore list.");
replaceDictionary = {};
# with open("replaceDictionary.dat","r") as fp:
# 	for line in fp:
# 		entry = line.strip().split("\t");
# 		if(len(entry)>1):
# 			replaceDictionary[entry[0]] = entry[1];

# ignoreSet = set();
# with open("ignoreSet.dat","r") as fp:
# 	for line in fp:
# 		ignoreSet.add(line.strip());


#Setting up nltk environment
if(verboseMode): print("Setting up nltk environment.");

Loading manual dictionary an ignore list.
Setting up nltk environment.


[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/acmbrito/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /home/acmbrito/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [7]:
%load_ext Cython

In [14]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/acmbrito/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [20]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/acmbrito/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [8]:
%%cython
from nltk.corpus import stopwords;
from nltk.stem.wordnet import WordNetLemmatizer;
import nltk.data;
from nltk.tokenize import word_tokenize, wordpunct_tokenize, sent_tokenize;
from nltk.corpus import wordnet;
import re
import nltk

lmtzr = WordNetLemmatizer();
sent_tokenizer=nltk.data.load('tokenizers/punkt/english.pickle')
punctuation = re.compile(r'[(\])(\})(\{)(\[).?!,":;()|]');
stopSet = set(stopwords.words('english'));


verboseMode = True;

# Loading manual dictionary an ignore list
if(verboseMode): print("Loading manual dictionary an ignore list.");
replaceDictionary = {};
with open("replaceDictionary.txt","r") as fp:
	for line in fp:
		entry = line.strip().split("\t");
		if(len(entry)>1):
			replaceDictionary[entry[0]] = entry[1];

ignoreSet = set();
with open("ignoreSet.txt","r") as fp:
	for line in fp:
		ignoreSet.add(line.strip());


#Setting up nltk environment
if(verboseMode): print("Setting up nltk environment.");


def findWholeWord(w):
	return re.compile(r'\b({0})\b'.format(w), flags=re.IGNORECASE).search

def get_wordnet_pos(treebank_tag):
	if treebank_tag.startswith('J'):
		return wordnet.ADJ
	elif treebank_tag.startswith('V'):
		return wordnet.VERB
	elif treebank_tag.startswith('N'):
		return wordnet.NOUN
	elif treebank_tag.startswith('R'):
		return wordnet.ADV
	else:
		return ''

#Setting up tokenizer
if(verboseMode): print("Setting up tokenizer.");

tokenizerInput = {
	"stopSet":stopSet,
	"punctuation":punctuation,
	"tokenizer":sent_tokenizer,
	"lematizer":lmtzr,
	"sent_tokenize": sent_tokenize,
	"replaceDictionary": replaceDictionary,
	"ignoreSet":ignoreSet
}

def tokenizeString(theString,maximumTokenSize,tokenizerInput,removeStopWords=True):
	stopSet = tokenizerInput["stopSet"];
	lematizer = tokenizerInput["lematizer"];
	tokenizer = tokenizerInput["tokenizer"];
	punctuation = tokenizerInput["punctuation"];
	sent_tokenize = tokenizerInput["sent_tokenize"];
	replaceDictionary = tokenizerInput["replaceDictionary"];
	ignoreSet = tokenizerInput["ignoreSet"];
	wordsList = [];
	titleAbstract = (". ".join(theString.split("::"))).strip();
	wordsSentences = [ word_tokenize(t) for t in sent_tokenize(titleAbstract)];
	stopSentence = False;
	for si, words in enumerate(wordsSentences):
		wordsTags = nltk.pos_tag(words);
		if(stopSentence):
			break;
		for wi,wordTag in enumerate(wordsTags):
			word = wordTag[0];
			tag = wordTag[1];
# 			if(si>len(wordsSentences)-4 and (word.lower()=="copyright" or (wi>0 and word.lower()=="c" and words[wi-1] == "("  and words[wi+1] == ")" ))):
# 				stopSentence = True;
# 				break;
			word = punctuation.sub("", word);
			convTag = get_wordnet_pos(tag);
			#print "w: "+word;
			if(convTag!=''):
				word  = lematizer.lemmatize(word.lower(),convTag);
			else:
				word  = lematizer.lemmatize(word.lower());
			if(len(word)==0 or ((word in stopSet) and removeStopWords) or (word in ignoreSet)):
				continue;
			else:
				if(word in replaceDictionary):
					wordsList.append(replaceDictionary[word]);
				else:
					wordsList.append(word);

	tokens = [set() for i in range(maximumTokenSize)];
	for wordIndex in range(len(wordsList)):
		for tokenSize in range(0,min(wordIndex+1,maximumTokenSize)):
			tokens[tokenSize].add(" ".join(wordsList[(wordIndex-tokenSize):(wordIndex+1)]));
	return tokens;

Loading manual dictionary an ignore list.
Setting up nltk environment.
Setting up tokenizer.


In [9]:
selectedJournal = 'coref_chu'
networksPath = ''

In [10]:
# Some util functions

def makeSurePathExists(path):
    try:
        os.makedirs(path)
    except OSError as exception:
        if exception.errno != errno.EEXIST:
            raise

# Parameters
jsonFileprefix = "%s_Comm"%selectedJournal;
removeStopWords = True;
maximumTokenSize = 2; #n-gram
minKeywordsPerCluster = 10;
maxKeywordsPerCluster = 10;
maxClusterNameLength = 150;
useMajorComponent = True;
verboseMode = True;

network = couplingNetwork
# Obtaining the major connected component (if needed)
if(useMajorComponent):
    if(verboseMode): print("Obtaining the major connected component.");
    network = network.clusters("WEAK").giant();

# Tokenizing the abstracts
if(verboseMode): print("Tokenizing the abstracts.");
tokensFrequency = [[] for i in range(maximumTokenSize)];
tokensGroupFrequency = [{} for i in range(maximumTokenSize)];

propertiesKeys = set();

verticesTokens = [];
for vertexIndex in range(network.vcount()):
    if(vertexIndex%100==0):
        print("Tokenizing: %d/%d             "%(vertexIndex,network.vcount()),end="\r")
        
#         for wordsList in tokenList:
#             tokens = [set() for i in range(maximumTokenSize)];
#             for wordIndex in range(len(wordsList)):
#                 for tokenSize in range(0,min(wordIndex+1,maximumTokenSize)):
#                     tokens[tokenSize].add(" ".join(wordsList[(wordIndex-tokenSize):(wordIndex+1)]));
    verticesTokens.append(tokenizeString(network.vs[vertexIndex]["abstract"],maximumTokenSize,tokenizerInput));

print("Done                   ");

# Obtaining the network community structure
if(verboseMode): print("Obtaining the network community structure.");

# cc = network.community_multilevel(weights="weight");
# communities = cc.membership;
network.get_edgelist()
edgelist = [(e.source,e.target) for e in network.es]
communities = infomapMembership(network.vcount(),edgelist)

# print("Modularity: %f"%cc.q);

clusters = [[] for i in range(max(communities)+1)];
for vertexIndex in range(network.vcount()):
    clusters[communities[vertexIndex]].append(vertexIndex);

#sorting the clusters by size
clusters = sorted(clusters,key=len,reverse=True);

# Getting tokens frequency
if(verboseMode): print("Getting tokens frequency.");




tokenFrequencyInClusters = [];
tokenFrequencyInCorpus = {};

for clusterIndex in range(len(clusters)):
    cluster = clusters[clusterIndex];
    tokenFrequencyInCluster = {};
    for vertexIndex in cluster:
        tokens = verticesTokens[vertexIndex];
        for tokenSize in range(0,maximumTokenSize):
            for token in tokens[tokenSize]:
                if(token not in tokenFrequencyInCorpus):
                    tokenFrequencyInCorpus[token] = 0;
                if(token not in tokenFrequencyInCluster):
                    tokenFrequencyInCluster[token] = 0;
                tokenFrequencyInCorpus[token] += 1;
                tokenFrequencyInCluster[token] += 1;
    tokenFrequencyInClusters.append(tokenFrequencyInCluster);
    
# Calculating the importance Index
if(verboseMode): print("Calculating the importance Index.");
#tokenRelativeFrequencyInClusters = [];
#tokenRelativeFrequencyOutClusters = [];
tokenImportanceIndexInClusters = [];

verticesCount = network.vcount();
for clusterIndex in range(len(clusters)):
    clusterSize = len(clusters[clusterIndex]);
    
    tokenFrequencyInCluster = tokenFrequencyInClusters[clusterIndex];
    
    #tokenRelativeFrequencyInCluster = {};
    #tokenRelativeFrequencyOutCluster = {};
    tokenImportanceIndexInCluster = {};
    
    for token in tokenFrequencyInCluster:
        nInCluster = tokenFrequencyInCluster[token];
        nOutCluster = tokenFrequencyInCorpus[token]-nInCluster;
        outClusterSize = verticesCount-clusterSize;
        if(nOutCluster==0):
            outClusterSize = 1; #Fix for singletons
        FInCluster = float(nInCluster)/float(clusterSize);
        FOutCluster = float(nOutCluster)/float(outClusterSize);
        importanceIndex = FInCluster-FOutCluster;
        #tokenRelativeFrequencyInCluster[token] = FInCluster;
        #tokenRelativeFrequencyOutCluster[token] = FOutCluster;
        tokenImportanceIndexInCluster[token] = importanceIndex;
    
    #tokenRelativeFrequencyInClusters.append(tokenRelativeFrequencyInCluster);
    #tokenRelativeFrequencyOutClusters.append(tokenRelativeFrequencyOutCluster);
    tokenImportanceIndexInClusters.append(tokenImportanceIndexInCluster);

defaultNames = "ABCDEFGHIJKLMNOPQRSTUWVXYZ";
defaultNamesLength = len(defaultNames);

clusterKeywords = [];
minClusterSize = min([len(cluster) for cluster in clusters]);
maxClusterSize = max([len(cluster) for cluster in clusters]);
clusterNames = [];
for clusterIndex in range(len(clusters)):
    cluster = clusters[clusterIndex];
    clusterSize = len(cluster);
    keywords = [v[0] for v in sorted(tokenImportanceIndexInClusters[clusterIndex].items(),key=operator.itemgetter(1),reverse=True)];
    if(maxClusterSize>minClusterSize):
        m = (maxKeywordsPerCluster-minKeywordsPerCluster)/float(maxClusterSize-minClusterSize);
    else:
        m=0;
    keywordsCount = round(m*(clusterSize-minClusterSize)+minKeywordsPerCluster);
    currentKeywords = [];
    while(len(currentKeywords)<keywordsCount and len(keywords)>len(currentKeywords)):
        currentKeywords = keywords[0:keywordsCount];
        jointKeywords = "."+".".join(currentKeywords)+".";
        toRemoveKeywords = [];
        for keyword in currentKeywords:
            if(jointKeywords.find(" %s."%keyword)>=0):
                toRemoveKeywords.append(keyword);
            elif(jointKeywords.find(".%s "%keyword)>=0):
                toRemoveKeywords.append(keyword);
        for toRemoveKeyword in toRemoveKeywords:
            keywords.remove(toRemoveKeyword);
            currentKeywords.remove(toRemoveKeyword);
    clusterKeywords.append(currentKeywords);
    #print(currentKeywords);
    clusterName = "";
    if(clusterIndex<defaultNamesLength):
        clusterName += defaultNames[clusterIndex];
    else:
        clusterName += "{%d}"%(clusterIndex);
    clusterName += " - "+", ".join(currentKeywords);
    if(len(clusterName)>maxClusterNameLength):
        clusterName = clusterName[0:maxClusterNameLength-1]+"...";
    for vertexIndex in cluster:
        network.vs[vertexIndex]["Cluster Name"] = clusterName;
        network.vs[vertexIndex]["Cluster Index"] = clusterIndex;
    clusterNames.append(clusterName);
    print(clusterName);


# Saving the network
if(verboseMode): print("Saving the network.");
# network.vs["kcore"] = network.coreness()

xn.igraph2xnet(network,fileName=PJ(networksPath,"%s.xnet"%jsonFileprefix),ignoredNodeAtts=["Text"]);

Obtaining the major connected component.
Tokenizing the abstracts.
Done                   92             
Obtaining the network community structure.
Getting tokens frequency.
Calculating the importance Index.
A - plasmonic, optical, resonance, nanoparticle, gold nanoparticles, nm, au, surface plasmon, particle, silver
B - device, two-dimensional, material, graphene oxide, sheet, electronic, application, 2d, property, mos2
C - cell, nanoparticles, cancer, delivery, drug, tumor, vivo, therapy, therapeutic, release
D - surfactant, micelle, concentration, solution, phase, aqueous, micellar, sodium, water, increase
E - surface, monolayer, spectroscopy, sam, gold, self-assembled monolayers, substrate, group, photoelectron, molecule
F - battery, capacity, cycle, electrochemical, performance, anode, lithium, g -1, high, electrode
G - organic, device, efficiency, polymer, film, solar cell, %, performance, power, poly
H - particle, colloidal, droplet, silica, emulsion, size, liquid, oil, suspens

In [19]:
network.vs.attributes()

['name', 'journal', 'year', 'Cluster Name', 'Cluster Index']

In [16]:
network.vs['Cluster Name'] = [c[:500] for c in network.vs['Cluster Name']]

In [13]:
del network.vs['refs']

In [17]:
xn.igraph2xnet(network,fileName=PJ(networksPath,"%s_v2.xnet"%jsonFileprefix),ignoredNodeAtts=["Text"]);

In [18]:
network.ecount()

52333760

In [20]:
network.vcount()

118292

In [23]:
network1 = network.simplify()

In [24]:
network1.ecount()

52333760